In [1]:
import numpy as np
import json
from transformers import AutoTokenizer
from random import randint

In [2]:
class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [3]:
def check_zero_rows_cols(permuted_array, sub_grid_x_dim, sub_grid_y_dim):
    reshaped_array = permuted_array.reshape((sub_grid_x_dim, sub_grid_y_dim))
    return np.any(np.all(reshaped_array == 0, axis=0)) or np.any(
        np.all(reshaped_array == 0, axis=1)
    )

In [4]:
# If num_prompts is 1, then we will print out the prompt. Otherwise, it will save a json file.
num_prompts = 100000
min_num_rows_and_columns = 6
max_num_rows_and_columns = 8

In [5]:
def make_move_obj_grids(min_grid_dim, max_grid_dim):
    input_grids = []
    output_grids = []

    min_move = -3
    max_move = 3
    random_move_x = randint(min_move, max_move)
    random_move_y = randint(min_move, max_move)
    if random_move_x == 0:
        while random_move_y == 0:
            random_move_y = randint(min_move, max_move)

    # Need 4 inputs, 4 outputs (3 train pairs, 1 test pair)
    pair_count = 0
    redo = False
    while pair_count < 4 or redo:
        redo = False
        
        grid_x_dim = randint(min_grid_dim, max_grid_dim)
        grid_y_dim = randint(min_grid_dim, max_grid_dim)
        
        min_sub_grid_dim = 2
        max_sub_grid_dim = 5
        sub_grid_x_dim = randint(min_sub_grid_dim, max_sub_grid_dim)
        sub_grid_y_dim = randint(min_sub_grid_dim, max_sub_grid_dim)

        size = sub_grid_x_dim * sub_grid_y_dim
        permuted_array = np.random.randint(10, size=size)

        if check_zero_rows_cols(permuted_array, sub_grid_x_dim, sub_grid_y_dim):
            redo = True
            continue

        isolated_obj_grid = np.zeros((grid_x_dim, grid_y_dim), dtype=int)
        
        
        # Place sub-grid randomly inside of isolated_obj_grid
        sub_grid = permuted_array.reshape((sub_grid_x_dim, sub_grid_y_dim))
        rand_start_x_index = randint(0, grid_x_dim - sub_grid_x_dim)
        rand_start_y_index = randint(0, grid_y_dim - sub_grid_y_dim)
        isolated_obj_grid[
            rand_start_x_index : rand_start_x_index + sub_grid_x_dim,
            rand_start_y_index : rand_start_y_index + sub_grid_y_dim,
        ] = sub_grid
        
        start_x = rand_start_x_index + random_move_x
        start_y = rand_start_y_index + random_move_y

        if start_x < 0 or start_y < 0:
            redo = True
        else:
            input_grid = np.copy(isolated_obj_grid)
            input_grids.append(input_grid)

            moved_obj_grid = np.zeros((2 * grid_x_dim, 2 * grid_y_dim), dtype=int)
            moved_obj_grid[
                start_x : start_x + sub_grid_x_dim,
                start_y : start_y + sub_grid_y_dim,
            ] = sub_grid

            moved_obj_grid = moved_obj_grid[:grid_x_dim, :grid_y_dim]

            output_grid = np.copy(moved_obj_grid)
            output_grids.append(output_grid)

            pair_count += 1
        
    return random_move_x, random_move_y, input_grids, output_grids

In [6]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

json_file = []
max_token_length = 0
for i in range(num_prompts):
    
    row_move, column_move, input_grids, output_grids = make_move_obj_grids(min_num_rows_and_columns, max_num_rows_and_columns)
    
    instruction = "Given the following input/output train pairs of ARCSolver grids: "
    
    train_input_grids = input_grids[:3]
    train_output_grids = output_grids[:3]
    
    for i, (train_input_grid, train_output_grid) in enumerate(zip(train_input_grids, train_output_grids)):
        instruction += (f"Train_Input_{i+1}=[")
        for j in range(len(train_input_grid)):
            instruction += "["
            for k in range(len(train_input_grid[j])):
                instruction += str(train_input_grid[j][k])
                if k != len(train_input_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(train_input_grid) - 1:
                instruction += ","
        instruction += "]"
        instruction += (f" and Train_Output_{i+1}=[")
        for j in range(len(train_output_grid)):
            instruction += "["
            for k in range(len(train_output_grid[j])):
                instruction += str(train_output_grid[j][k])
                if k != len(train_output_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(train_output_grid) - 1:
                instruction += ","
        instruction += "]"
        if i != len(train_output_grids) - 1:
            instruction += ", "
            
    test_input_grids = input_grids[3:]
    test_output_grids = output_grids[3:]
    
    instruction += ". Find the transformation from each input grid to output grid that is common to all 3 train pairs."
    instruction += " Then apply this transformation to the following test input grid to get the test output grid: "
        
    
    for i, (test_input_grid) in enumerate(test_input_grids):
        instruction += (f"Test_Input_{i+1}=[")
        for j in range(len(test_input_grid)):
            instruction += "["
            for k in range(len(test_input_grid[j])):
                instruction += str(test_input_grid[j][k])
                if k != len(test_input_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(test_input_grid) - 1:
                instruction += ","
        instruction += "]"
    instruction += "."
    
    output = "The common transformation is that the non-zero element sub-grid in each train input grid is moved " + str(row_move) +\
        " units vertically and " + str(column_move) + " units horizontally to get the corresponding train output grid. Therefore, "
    for i, (test_output_grid) in enumerate(test_output_grids):
        output += (f"Test_Output_{i+1}=[")
        for j in range(len(test_output_grid)):
            output += "["
            for k in range(len(test_output_grid[j])):
                output += str(test_output_grid[j][k])
                if k != len(test_output_grid[j]) - 1:
                    output += ","
            output += "]"
            if j != len(test_output_grid) - 1:
                output += ","
        output += "]"
        if i != len(test_output_grids) - 1:
            output += ", "
    output += "."
     
    prompt = instruction + " " + output

    # Tokenize the request text
    tokenized_request = tokenizer.tokenize(prompt)

    # Get the token length
    token_length = len(tokenized_request)
    if token_length > max_token_length:
        max_token_length = token_length

    json_file.append({"instruction": instruction, "output": output})

    if num_prompts == 1:
        print("Instruction:")
        print(instruction)
        print("\nOutput:")
        print(output)

if num_prompts > 1:
    json_string = json.dumps(json_file, cls=NumpyArrayEncoder)
    base_file_name = "../data/ARCSolver_move_obj_puzzles_" + str(num_prompts)
    filename = base_file_name + ".json"
    filepath = filename
    with open(filepath, "w") as outfile:
        outfile.write(json_string)

print(f"\nMax Token Length: {max_token_length}")


Max Token Length: 1236
